# Usando la API de DeepSeek - Reasoner
https://api-docs.deepseek.com/guides/reasoning_model


 🔍 **¿Qué es DeepSeek Reasoner?**  
DeepSeek es una plataforma de IA avanzada diseñada para ofrecer soluciones inteligentes y personalizadas en tiempo real.  
Con modelos de lenguaje de última generación, DeepSeek te permite:  

DeepSeek Reasoning Model es un modelo avanzado de razonamiento y generación de lenguaje desarrollado por DeepSeek, 
diseñado para realizar tareas complejas que requieren comprensión profunda, lógica y capacidad de inferencia.  
A diferencia de los modelos de lenguaje tradicionales, está optimizado para:

✅ Razonamiento paso a paso  
✅ Alta precisión en tareas complejas  
✅ Contexto extendido (hasta 128K tokens)  
✅ Optimización para respuestas estructuradas    

In [ ]:
# Importa el cliente estandar de OpenAI, se conecta DIRECTAMENTE a los servicios de OpenAI como : GPT-3, GPT-4, DALL-E, etc.
from openai import OpenAI
# Librerias que permite interactuar con el Sistema Operativo. Administrar archivos, variables de entorno y más.
import os

In [ ]:
os.environ["DEEPSEEK_API_KEY"] = "sk-ebed09dc4b2242bebd1e21cd3be686d3"  # ¡Mantén tus claves seguras!

### Deepseek-Chat
Diseñado para respuestas rápidas y directas, ideal para interacciones conversacionales estándar.

In [ ]:
client = OpenAI(
    api_key=os.environ["DEEPSEEK_API_KEY"],  # Clave API de OpenAI.
    base_url="https://api.deepseek.com"  # URL base de la API. Por defecto https://api.openai.com/v1, puedes cambiarla si estás utilizando otra solucion.
    )

# El metodo client.chat.completions.create es una funcion proporcionada por la biblioteca OpenAI
# Facilita la interaccion con modelos de lenguaje basados en chat, como GPT-4 u otros. 
# Permite enviar una solicitud a la API de OpenAI (o a una API compatible, como DeepSeek) para generar respuestas de chat en formato de conversacion.
response = client.chat.completions.create(
    model="deepseek-chat",  # Nombre del modelo a utilizar como: "gpt-4-turbo", "deepseek-chat"
    # Lista de mensajes del chat en formato [{role: "user", content: "Hola"}]
    messages=[  
        {"role": "system", "content": "Eres un profesor de matematicas"},
        {"role": "user", "content": "9.11 o 9.8, cual es mayor?"}
    ],    
    stream=False,  # (True) la respuesta del modelo se envía en fragmentos (streaming) y (False) la respuesta del modelo como un bloque completo.
    #max_tokens= 50,  # Límite máximo de tokens en la respuesta generada.
    temperature = 0.7,  # Controla la aleatoriedad. Rango (0 a 2). Respuestas más deterministas (0.0 - 0.3) , respuestas muy aleatorias (1.3 - 2.0).
    timeout=30 # Cancela la conexión si transcurre 30 segundos sin recibir la respuesta completa.
)

resultado = response.choices[0].message
print("Respuesta:", resultado.content)
print("--------------------------------------------")
print("Tokens usados:", response.usage.total_tokens)

### Deepseek-Reasoner
Optimizado para tareas que requieren un análisis más profundo y detallado, lo que resulta en tiempos de respuesta más largos.

In [ ]:
client = OpenAI(
    api_key=os.environ["DEEPSEEK_API_KEY"], 
    base_url="https://api.deepseek.com"
    )

messages = [{"role": "system", "content": "Eres un profesor"},
            {"role": "user", "content": "9.11 o 9.8, cual es mayor?"}
           ]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages= [
        {"role": "system", "content": "Eres un profesor"},
        {"role": "user", "content": "9.11 o 9.8, cual es mayor?"}
],
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

In [ ]:
print("Reasoning:", reasoning_content)

In [ ]:
print("Respuesta:", content)

#### Streaming

In [ ]:
client = OpenAI(
    api_key=os.environ["DEEPSEEK_API_KEY"], 
    base_url="https://api.deepseek.com"
    )

messages = [{"role": "system", "content": "Eres un profesor"},
            {"role": "user", "content": "9.11 o 9.8, cual es mayor?"}
           ]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)

print("========== RAZONAMIENTO EN STREAMING =============")
respuesta_actual = ""  # Acumulador para la respuesta

for chunk in response:
    if not chunk.choices:
        continue
    
    delta = chunk.choices[0].delta
    
    if getattr(delta, 'reasoning_content', None):
        print(delta.reasoning_content, end='', flush=True)
    
    if getattr(delta, 'content', None):
        respuesta_actual += delta.content  # Acumula sin imprimir

# Imprime la respuesta completa al final
if respuesta_actual:
    print(f"\n\n========== RESPUESTA ==========\n{respuesta_actual}")